In [ ]:
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Классификация с помощью PyTorch

In [ ]:
iris = load_iris() # импортируем данные ирисов
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42) # делим данные на привычные наборы

In [ ]:
X_train = torch.tensor(X_train).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
# присваиваем выборки в тензоры для работы с PyTorch
# нормализация данных
mean = X_train.mean(dim=0)
std = X_train.std(dim=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [ ]:
model = torch.nn.Sequential( # строим модель классификации
    torch.nn.Linear(in_features = 4, out_features =3),
    torch.nn.Softmax(dim=1))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# обучаем модель, задаём 1000 эпох
num_epochs = 1000
for epoch in range(num_epochs):

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.6551
Epoch [200/1000], Loss: 0.6503
Epoch [300/1000], Loss: 0.6461
Epoch [400/1000], Loss: 0.6424
Epoch [500/1000], Loss: 0.6390
Epoch [600/1000], Loss: 0.6360
Epoch [700/1000], Loss: 0.6332
Epoch [800/1000], Loss: 0.6307
Epoch [900/1000], Loss: 0.6285
Epoch [1000/1000], Loss: 0.6264


In [ ]:
with torch.no_grad(): # оцениваем модель
    y_pred = model(X_test)
    _, predicted = torch.max(y_pred, dim=1)
    accuracy = (predicted == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

Test Accuracy: 1.0000


# Регрессия с помощью PyTorch

In [ ]:
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
wine_url = "https://nagornyy.me/datasets/wine_reviews.csv.zip"
df_wine = pd.read_csv(wine_url)
# импортируем данные вин, удаляем вина без рейтинга или без цены
df_wine.dropna(subset=["points", "price"], inplace=True)
df_wine.shape

(18198, 10)

In [ ]:
df_wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"With a delicate, silky mouthfeel and bright ac...",NaN,86,23.0,California,Central Coast,Central Coast,Pinot Noir,MacMurray Ranch
1,Italy,D'Alceo is a drop dead gorgeous wine that ooze...,D'Alceo,96,275.0,Tuscany,Toscana,NaN,Red Blend,Castello dei Rampolla
2,France,The great dominance of Cabernet Sauvignon in t...,NaN,91,40.0,Bordeaux,Haut-Médoc,NaN,Bordeaux-style Red Blend,Château Bernadotte
3,Italy,"The modest cherry, dark berry and black tea no...",NaN,81,15.0,Tuscany,Chianti Classico,NaN,Sangiovese,Valiano
4,US,"Exceedingly light in color, scent and flavor, ...",NaN,83,25.0,Oregon,Rogue Valley,Southern Oregon,Pinot Noir,Deer Creek


In [ ]:
y = df_wine.points.values
X = np.log(df_wine.price.values) # обозначаем зависимую и независимую переменные

In [ ]:
y_tensor = torch.from_numpy(y.reshape(-1, 1)).float()
X_tensor = torch.from_numpy(X.reshape(-1, 1)).float() # присваиваем переменные в тензоры для работы с PyTorch

In [ ]:
def lin_reg_model(X, w, a): # определяем функцию линейной регрессии
    return X @ w.t() + a

def mse(true, predicted): # считаем оценку модели
    return ((true - predicted) ** 2).sum() / true.numel()

In [ ]:
# задаём начальные веса параметров, чтобы найти минимум функции потерь
weight = torch.randn(y_tensor.shape[1], X_tensor.shape[1]) / math.sqrt(2 / y_tensor.shape[1])
bias = torch.zeros(y_tensor.shape[1])
# начинаем записывать историю вычислений
weight.requires_grad_(True)
bias.requires_grad_(True);

In [ ]:
learning_rate = 0.01
epochs = 10000

for epoch in range(epochs):
    # вычисляем функцию потерь
    predictions = lin_reg_model(X_tensor, weight, bias)
    loss = mse(predictions, y_tensor)
    loss.backward()
    # вычитаем производные из параметров
    # записывать историю вычислений уже не нужно (no_grad)
    with torch.no_grad():
        weight -= weight.grad * learning_rate
        bias -= bias.grad * learning_rate
        # обнуляем производные
        weight.grad.zero_()
        bias.grad.zero_()
    # выводим значение фукнции потерь каждые 2000 повторов
    if (epoch + 1) % (epochs / 10) == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 999, loss 53.415122985839844
epoch 1999, loss 17.276302337646484
epoch 2999, loss 9.018627166748047
epoch 3999, loss 7.1317291259765625
epoch 4999, loss 6.700572490692139
epoch 5999, loss 6.602056980133057
epoch 6999, loss 6.5795440673828125
epoch 7999, loss 6.574398517608643
epoch 8999, loss 6.573220729827881
epoch 9999, loss 6.57295036315918
